---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [93]:
import pandas as pd
from datetime import datetime
import re

In [119]:
def date_sorter():
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.DataFrame(doc,columns=['raw'])
    #截取原始date資料
    df['date_type1'] = df['raw'].str.findall(r'\d?\d[-/]\d?\d[-/]\d{2,4}')
    df['date_type2'] = df['raw'].str.findall(r'(?:\d?\d[-, ]*)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z,.]*(?:[ ]\d?\d[a-z-, ]*)?[- ]*\d{2,4}')
    df['date_type3'] = df['raw'].str.findall(r'\d?\d[-/]\d{4}')
    df['date_type4'] = df['raw'].str.findall(r'\d{4}')
    df['final_date'] = 0
    #date轉換為可計算的datetime
    p = {'Jan':1,'January':1,'Janaury':1,'Feb':2,'February':2,'Mar':3,'March':3,'Apr':4,'April':4,'May':5,'Jun':6,'June':6,
     'Jul':7,'July':7,'Aug':8,'August':8,'Sep':9,'September':9,'Oct':10,'October':10,'Nov':11,'November':11,'Dec':12,'December':12,'Decemeber':12}
    for i in df.index:
        if df.loc[i,'date_type1']==[]:
            if df.loc[i,'date_type2']==[]:
                if df.loc[i,'date_type3']==[]:
                    df.loc[i,'final_date'] = datetime(int(df.loc[i,'date_type4'][0]),1,1)
                else:
                    date = df.loc[i,'date_type3'][0].split('/')
                    df.loc[i,'final_date'] = datetime(int(date[1]),int(date[0]),1)
            else:
                date = re.split(r'[,. ]+',df.loc[i,'date_type2'][0])
                if date[0].isalpha() and len(date)==3:
                    date[0] = p.get(date[0])
                    df.loc[i,'final_date'] = datetime(int(date[2]),int(date[0]),int(date[1]))
                elif date[0].isalpha() and len(date)==2:
                    date[0] = p.get(date[0])
                    df.loc[i,'final_date'] = datetime(int(date[1]),int(date[0]),1)
                else:
                    if len(date)==2:
                        date.append(date[0][0])
                        date.append(date[0][1:])
                        date[0] = p.get(date[3])
                        df.loc[i,'final_date'] = datetime(int(date[1]),int(date[0]),int(date[2]))
                    else:
                        date[1] = p.get(date[1])
                        df.loc[i,'final_date'] = datetime(int(date[2]),int(date[1]),int(date[0]))
        else:
            date = re.split(r'[/-]+',df.loc[i,'date_type1'][0])
            if int(date[2])<100:
                date[2] = '19' + date[2]
                df.loc[i,'final_date'] = datetime(int(date[2]),int(date[0]),int(date[1]))
            else:
                df.loc[i,'final_date'] = datetime(int(date[2]),int(date[0]),int(date[1]))
    #轉換成答案要求的series
    ans = pd.Series(list(df.sort_values('final_date').index))
    return ans